# Ray Concepts - Data Parallelism (Part 2)

The previous lesson explored Ray's core concepts and how they work. We learned how to define Ray _tasks_, run them, and retrieve the results. We  also started learning about how Ray schedules tasks in a distributed environment.

This lesson completes the discussion of Ray tasks by exploring how task dependencies are handled. We'll also look under the hood at Ray's architecture and runtime behavior.

> **Tip:** Recall that the [Ray Package Reference](https://ray.readthedocs.io/en/latest/package-ref.html) in the [Ray Docs](https://ray.readthedocs.io/en/latest/) is useful for exploring the API features we'll learn.

In [90]:
# Imports and initialize Ray. We're adding NumPy for the examples and the tutorial `util` library:

import ray, time, sys    # New notebook, so new process
import numpy as np       # Used for examples
sys.path.append('..')    # Import our own libraries starting in the project root directory

from util.printing import p

In [91]:
ray.init(ignore_reinit_error=True)

2020-04-06 08:26:29,309	ERROR worker.py:682 -- Calling ray.init() again after it has already been called.


Let's work with a new remote function. Previously, our `expensive` and `expensive_task` functions returned tuples that included time durations. Obviously the durations were useful for understanding how long the functions took to execute. Now, it will be more convenient to not return "metadata" like this, but just data values that we care about, because we are going to pass them to other functions. 

Hence, we'll define _dependentcy_ relationships between tasks. We'll learn how Ray handles these dependent, asynchronous computations.

So, let's define a task to return a random NumPy array of some size `n`. As before, we'll add a sleep time, one tenth the size of `n`:

In [101]:
@ray.remote
def make_array(n):
    time.sleep(n/10.0)
    return np.random.standard_normal(n)

Now define a task that can add two NumPy arrays together. The arrays need to be the same size, but we'll ignore any checking for this requirement.

In [102]:
@ray.remote
def add_array(a1, a2):
    time.sleep(a1.size/10.0)
    return np.add(a1, a2)

Now lets use them!

In [104]:
start = time.time()
id1 = make_array.remote(20)
id2 = make_array.remote(20)
id3 = add_array.remote(id1, id2)
print(ray.get(id3))
p(0, time.time() - start)  # The integer argument to `p()` is arbitrary.

[-3.1179096   1.96746457 -0.53631989  0.4197659   0.93568958  0.54379198
  1.420145    1.3533405   0.79262948  1.54804509  0.02092625 -2.35780908
 -0.28825194 -3.27000509 -0.40365204  0.1378132  -1.03654836 -0.13040567
 -0.85492694 -0.52611309]
 0:  4.018 seconds


Something subtle and "magical" happened here. Note that when we called `add_array`, we didn't need to call `ray.get()` first for `id1` and `id2`, since `add_array` expects NumPy arrays. Because `add_array` is a Ray task, Ray automatically does the extraction for us, so we can write code that looks more natural.

Furthermore, note that the `add_array` task effectively depends on the outputs of the two `make_array` tasks. Ray won't run `add_array` until the other tasks are finished. Hence, Ray handles task dependencies automatically for us. 

This is why the elapsed time is about 4 seconds. We used a size of 20, so we slept 2 seconds in each call to `make_array`, but those happened in parallel, _followed_ by a second sleep of 2 seconds in `add_array`.

Recall from the previous lesson that we explored when to call `ray.get()` to avoid forcing your tasks to become synchronous when they should be asynchronous. This additional example illustrates two key points:

* _Don't ask for results you don't need_
* _Don't ask for the results you need until you really need them_

We don't need to see the objects for `id1` and `id2`. We only need the final array for `id3`.

## Ray.wait() - A Better Alternative than ray.get()

We've seen several examples of the best idiomatic way to use ray.get(). Here again is an example from the last lesson:

```python
start_all = time.time()
ids = [expensive_task.remote(n) for n in range(5)]  # Fire off the asynchronous tasks
for n2, duration in ray.get(ids):                   # Retrieve all the values from the list of futures
    p(n2, duration)
print("Total time:")
p(10, time.time() - start_all)
```

Let's try it again with our new `make_array()` method:

In [109]:
start_all = time.time()
array_ids = [make_array.remote(n*10) for n in range(5)]
added_array_ids = [add_array.remote(id, id) for id in array_ids]
for array in ray.get(added_array_ids):
    print(f'{array.size}: {array}')
print("Total time:")
p(0, time.time() - start_all)

0: []
10: [-2.05717767 -1.21196527  0.53912555  1.16772308 -1.58010648 -0.48667315
 -0.50219433 -2.67222693  2.75465449 -1.53238876]
20: [-0.2228689   1.28494684  1.80445664 -0.00814641 -2.81902369  0.4593267
 -1.39633369  0.16244171 -0.45688638 -0.76981611  1.45178854  0.33817777
  1.05311923  2.66496464 -1.09220489  2.2759349  -1.3036437  -0.3377501
  0.80299418  0.20212094]
30: [-1.45446997  1.63835661 -2.67604949 -0.50339876 -0.43857409 -0.18405963
 -1.19022442  0.21649415 -3.78545935  0.51321947  0.80649473  0.09703079
  1.19964202 -0.72702715  0.0610685  -2.03076987  3.25233161  0.33781107
 -5.39413574  4.78305037  1.44991144  3.17309371  1.03222492 -3.27753871
 -0.51567295 -1.55897283  1.7056837  -2.17719916 -0.26781733  1.62440777]
40: [ 0.9426931   0.14557628 -0.79470241 -2.95832785  2.05747984  2.37633456
  0.27908318  1.44874428  2.08220845 -2.94162102 -2.06906131  3.33153504
 -0.16998523 -2.14003902  2.16711531  2.17519085  1.79551497 -1.76010543
 -0.28754492  1.87435363  1

On my machine, I waited 8 seconds and then everything was printed at once.

There are two fundamental problems with `ray.get()`:

1. There's no timeout, in case something gets "hung".
2. We have to wait for _all_ the objects to be available before `ray.get()` returns.

The ability to specify a timeout is essential in production code as a defensive measure. Many potential problems could happen in a real production system, any one of which could cause the task we're waiting on to take an abnormally long time to complete or never complete. Our application would be deadlocked waiting on this task. Hence, it's **strongly recommended** in production software to always use timeouts on blocking calls, so that the application can attempt some sort of recovery in situations like this, or at least report the error and "degrade gracefully".

Actually, there _is_ a `timeout=<value>` option you can pass to `ray.get()` ([documentation](https://ray.readthedocs.io/en/latest/package-ref.html#ray.get)), but it will most likely be removed in a future release of Ray. Why remove it if timeouts are important? This change will simplify the implementation of `ray.get()` and encourage the use of `ray.wait()` instead, which currently and always will support a `timeout` optional parameter. This will be the "one way to do things", where blocking with timeouts is concerned.

Using `ray.wait()` is also the way to fix the second problem with `ray.get()`, that we have to wait for all tasks to finish before we get any values back. Some of those tasks might finish quickly, like our contrived examples that sleep for short durations compared to other invocations. 

When you have a list of asynchronous tasks, you want to process the results of them as soon they finish, even while others continue to run. Use `ray.wait()` for this purpose.

Therefore, while `ray.get()` is simple and convenient, for _production code_, we recommend using `ray.wait()`, **with** timeouts. Now we'll learn how to use it effectively. For a longer discussion on `ray.wait()`, see [this blog post](https://medium.com/distributed-computing-with-ray/ray-tips-and-tricks-part-i-ray-wait-9ed7a0b9836d).

## How Distributed Task Management Works

> **Note:** If you just want to learn the Ray API, you can safely skip the rest of this lesson (notebook) for now. It continues the exploration of how Ray works internally, which we started in the previous lesson. However, you should come back to this material at some point, so you'll develop a better understanding of how Ray works.